In [16]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 28 12:10:04 2023

@author: gordon.donald
"""

#Get MSOA to BUA lookup
#We're using the overlap method. All MSOAs which overlap the BUA will be included


import git
import os
repo = git.Repo('.', search_parent_directories=True)
os.chdir(repo.working_tree_dir)
import sys
sys.path.append(repo.working_tree_dir)

import pandas as pd
import geopandas as gpd
from analysis.utils import *


In [ ]:

BUAs = gpd.read_file(r"Q:\SDU\Towns_indicative_selection_QA\Shapefiles\BUA_2022_GB_2824430073212747649\BUA_2022_GB.shp")
#THIS IS JUST GB

In [18]:
BUAs


,BUA22CD,BUA22NM,BUA22NMW,BUA22NMG,BNG_E,BNG_N,LONG,LAT,geometry
0,E63000001,Berwick-upon-Tweed,NaN,NaN,399257.449251,652237.077568,-2.013401,55.763482,"MULTIPOLYGON (((398500.000 652075.000, 398525...."
1,E63000002,Loanend,NaN,NaN,395742.427351,650004.527509,-2.069382,55.743404,"POLYGON ((395550.000 650000.000, 395550.000 64..."
2,E63000003,Norham,NaN,NaN,389892.097658,647250.813580,-2.162463,55.718573,"MULTIPOLYGON (((389875.000 647500.000, 389900...."
3,E63000004,Haggerston,NaN,NaN,404604.408475,643967.137732,-1.928327,55.689156,"MULTIPOLYGON (((403950.000 643675.000, 403925...."
4,E63000005,Lowick,NaN,NaN,401794.049653,639630.346258,-1.973055,55.650207,"POLYGON ((401125.000 639725.000, 401125.000 63..."
...,...,...,...,...,...,...,...,...,...
8540,W45000618,St Donat's,Sain Dunwyd,NaN,293609.517719,168185.710715,-3.530852,51.402690,"POLYGON ((293325.000 167825.000, 293325.000 16..."
8541,W45000619,St Athan,Sain Tathan,NaN,301644.561751,168082.188325,-3.415351,51.403211,"MULTIPOLYGON (((302325.000 168400.000, 302325...."
8542,W45000620,East Aberthaw,NaN,NaN,303425.807752,167183.715163,-3.389507,51.395441,"MULTIPOLYGON (((303125.000 167325.000, 303100...."
8543,W45000621,Rhoose,Y Rhws,NaN,306199.083615,167107.597500,-3.349637,51.395223,"MULTIPOLYGON (((305075.000 166300.000, 305075...."


In [19]:

BUA_code = "E63000001"

def get_msoas_in_BUA(BUA_code):
    shape = BUAs[BUAs.BUA22CD == BUA_code]
    msoas_within = list(gpd.overlay(msoas, shape)['msoa11cd'])
    return(msoas_within)

def get_lsoas_in_BUA(BUA_code):
    shape = BUAs[BUAs.BUA22CD == BUA_code]
    lsoas_within = list(gpd.overlay(lsoas, shape)['lsoa11cd'])
    return(lsoas_within)



In [20]:

GET_MSOA_BUA_LOOKUP = True
GET_LSOA_BUA_LOOKUP = True


In [21]:
#For each MSOA, which BUA is it in?
if GET_MSOA_BUA_LOOKUP or GET_LSOA_BUA_LOOKUP:

    #We also need Greater London for this, which isn't in BUAs methodology.
    #regions = gpd.read_file("Q:\GI_Data\Boundaries\Regions\Regions2019.shp")
    #gl = regions.iloc[6:7]
    #gl.columns = ['id', 'BUA22CD', 'BUA22NM', 'BNG_E', 'BNG_N', 'LONG', 'LAT', 'AREA', 'LEN', 'geometry']

    #BUAs = pd.concat([BUAs, gl])

    #Get BUA populations, to select largest BUA
    #NO Northern Ireland at the moment
    bua_pop = pd.read_excel("Q:/SDU/Towns_indicative_selection_QA/Data/OFF-SEN - BUA Analysis - LUWPs Ranking v01.02.xlsx", sheet_name='BUA_Population', skiprows=2)
    bua_pop_wales = pd.read_csv("Q:/SDU/Towns_indicative_selection_QA/Data/BUA_pop-wales.csv", skiprows=2, encoding = 'unicode_escape') #Wales sheet of this data)
    bua_pop_scotland = pd.read_csv("Q:/SDU/Towns_indicative_selection_QA/Data/Scotland_settlement_pop.csv")
    bua_pop_london=pd.read_csv(r"Q:\SDU\Mobility\Data\Processed\LondonBUApop2021_FromNomis.csv")
    bua_pop = bua_pop[['BUA name', 'BUA code', 'Counts']]
    bua_pop_wales = bua_pop_wales[['BUA name', 'BUA code', ' Counts ']]
    bua_pop_scotland = bua_pop_scotland[['Settlement name', 'Settlement code', 'population']]


    bua_pop.columns = ['BUA22NM', 'BUA22CD', 'Population']
    bua_pop_wales.columns = ['BUA22NM', 'BUA22CD', 'Population']
    bua_pop_scotland.columns = ['BUA22NM', 'BUA22CD', 'Population']
    bua_pop_london.columns=['BUA22NM', 'BUA22CD', 'Population']
    bua_pop_all = pd.concat([bua_pop, bua_pop_scotland, bua_pop_wales, bua_pop_london])
    print(bua_pop_all.columns)

    #gl_pop = pd.DataFrame(['London', 'E12000007', 9468000]).T
    
    #gl_pop.columns=bua_pop_all.columns

    #bua_pop_all = pd.concat([bua_pop_all, gl_pop])

    BUAs = BUAs.merge(bua_pop_all, left_on='BUA22NM', right_on='BUA22NM')
    
    #BUAs.to_file('Q:/SDU/Mobility/Data/Processed/BUAs4.shp')


Index(['BUA22NM', 'BUA22CD', 'Population'], dtype='object')


In [22]:
if True:
    if GET_MSOA_BUA_LOOKUP:
        
        msoas=get_all_small_areas()
        
        bua_by_msoa = gpd.overlay(BUAs, msoas)
        bua_by_msoa_largest_BUA = bua_by_msoa.sort_values('Population').drop_duplicates(['msoa11cd'], keep='last')
    
        bua_by_msoa_to_store = bua_by_msoa_largest_BUA[['msoa11cd', 'msoa11nm', 'BUA22CD_y', 'BUA22NM', 'Population']]
        bua_by_msoa_to_store.columns = ['msoa11cd', 'msoa11nm', 'BUA22CD', 'BUA22NM', 'Population']
    
        not_in_BUA_msoa = msoas[~msoas.msoa11cd.isin(bua_by_msoa_largest_BUA.msoa11cd)]
        not_in_BUA_msoa_to_store = not_in_BUA_msoa[['msoa11cd', 'msoa11nm']]
        not_in_BUA_msoa_to_store['BUA22CD'] = 'None'
        not_in_BUA_msoa_to_store['BUA22NM'] = 'None'
        not_in_BUA_msoa_to_store['Population'] = 0
        
        bua_by_msoa_to_store = pd.concat([bua_by_msoa_to_store, not_in_BUA_msoa_to_store])   
        
        bua_by_msoa_to_store.sort_values('BUA22CD').to_csv("Q:/SDU/Mobility/Data/Processed/MSOA_to_BUA__NOGLA_lookup.csv", index=False, mode='w')


D:\Users\ggliozzo\AppData\Local\Temp\1\ipykernel_8352\1934558341.py:6: UserWarning: `keep_geom_type=True` in overlay resulted in 884 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  bua_by_msoa = gpd.overlay(BUAs, msoas)


In [24]:
if True:
    if GET_LSOA_BUA_LOOKUP:
        
        lsoas=get_all_tiny_areas()
        
        bua_by_lsoa = gpd.overlay(BUAs, lsoas, keep_geom_type=False)
        bua_by_lsoa_largest_BUA = bua_by_lsoa.sort_values('Population').drop_duplicates(['lsoa11cd'], keep='last')
    
        bua_by_lsoa_to_store = bua_by_lsoa_largest_BUA[['lsoa11cd', 'lsoa11nm', 'BUA22CD_y', 'BUA22NM', 'Population']]
        bua_by_lsoa_to_store.columns = ['lsoa11cd', 'lsoa11nm', 'BUA22CD', 'BUA22NM', 'Population']
    
        not_in_BUA_lsoa = lsoas[~lsoas.lsoa11cd.isin(bua_by_lsoa_largest_BUA.lsoa11cd)]
        not_in_BUA_lsoa_to_store = not_in_BUA_lsoa[['lsoa11cd', 'lsoa11nm']]
        not_in_BUA_lsoa_to_store['BUA22CD'] = 'None'
        not_in_BUA_lsoa_to_store['BUA22NM'] = 'None'
        not_in_BUA_lsoa_to_store['Population'] = 0
        
        bua_by_lsoa_to_store = pd.concat([bua_by_lsoa_to_store, not_in_BUA_lsoa_to_store])   
        
        bua_by_lsoa_to_store.sort_values('BUA22CD').to_csv("Q:/SDU/Mobility/Data/Processed/LSOA_to_BUA_NOGLA_lookup.csv", index=False)
